# Demo

Injection of first order logic rules into a neural network for iris classification task.

Some imports.

In [1]:
from keras import Input, Model
from psyki import Injector
from psyki.fol import Parser
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from test import get_mlp
from test import get_rules
import tensorflow as tf
from keras.optimizer_v2.adam import Adam

Loading iris dataset and separation into train and test set.

In [2]:
x, y = load_iris(return_X_y=True, as_frame=True)
encoder = OneHotEncoder(sparse=False)
encoder.fit_transform([y])
dataset = x.join(y)

train, test = train_test_split(dataset, test_size=0.5, random_state=0)
train

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
3,4.6,3.1,1.5,0.2,0
149,5.9,3.0,5.1,1.8,2
98,5.1,2.5,3.0,1.1,1
6,4.6,3.4,1.4,0.3,0
68,6.2,2.2,4.5,1.5,1
...,...,...,...,...,...
9,4.9,3.1,1.5,0.1,0
103,6.3,2.9,5.6,1.8,2
67,5.8,2.7,4.1,1.0,1
117,7.7,3.8,6.7,2.2,2


Import FOL rules:

- PL <= 2.28 <- X = setosa
- PL > 2.28 ^ PW > 1.64 <- X = virginica
- PL > 2.28 ^ PW <= 1.64 <- X = versicolor

In [3]:
features_mapping = {
    'SL': 0,
    'SW': 1,
    'PL': 2,
    'PW': 3,
}
class_mapping = {
    'setosa': tf.constant([1, 0, 0], dtype=tf.float32),
    'virginica': tf.constant([0, 1, 0], dtype=tf.float32),
    'versicolor': tf.constant([0, 0, 1], dtype=tf.float32)
}

parser = Parser.default_parser()
iris_rules = [parser.get_function(rule, features_mapping, class_mapping)
               for _, rule in get_rules('iris').items()]

Injection of fuzzy logic function derived from FOL rules into a neural network.

In [4]:
input_features = Input((4,), name='Input')
network = get_mlp(input=input_features, output=3, layers=3, neurons=64, activation_function='relu',
                  last_activation_function='softmax')
injector = Injector(network, input_features, output_shape=3)
injector.inject(iris_rules)
new_network = injector.predictor
new_network.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
new_network.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, 4)]          0                                            
__________________________________________________________________________________________________
L_1 (Dense)                     (None, 64)           320         Input[0][0]                      
__________________________________________________________________________________________________
L_2 (Dense)                     (None, 64)           4160        L_1[0][0]                        
__________________________________________________________________________________________________
L_3 (Dense)                     (None, 3)            195         L_2[0][0]                        
______________________________________________________________________________________________

Training.

In [5]:
train_x, train_y = train.iloc[:,:-1], train.iloc[:,-1]
new_network.fit(train_x, train_y, verbose=1, batch_size=5, epochs=30)

Epoch 1/30


2022-01-17 15:05:56.755003: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-01-17 15:05:56.755152: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


15/15 [==============================] - 1s 1ms/step - loss: 0.9155 - accuracy: 0.4400
Epoch 2/30
15/15 [==============================] - 0s 950us/step - loss: 0.6421 - accuracy: 0.9467
Epoch 3/30
15/15 [==============================] - 0s 950us/step - loss: 0.4910 - accuracy: 0.9467
Epoch 4/30
15/15 [==============================] - 0s 886us/step - loss: 0.3808 - accuracy: 0.9600
Epoch 5/30
15/15 [==============================] - 0s 891us/step - loss: 0.3234 - accuracy: 0.9467
Epoch 6/30
15/15 [==============================] - 0s 834us/step - loss: 0.2921 - accuracy: 0.9467

Removing the injected rules from the network.

In [6]:
new_network = Model(new_network.input, new_network.layers[-3].output)
new_network.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
new_network.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 4)]               0         
_________________________________________________________________
L_1 (Dense)                  (None, 64)                320       
_________________________________________________________________
L_2 (Dense)                  (None, 64)                4160      
_________________________________________________________________
L_3 (Dense)                  (None, 3)                 195       
Total params: 4,675
Trainable params: 4,675
Non-trainable params: 0
_________________________________________________________________


Evaluation.

In [7]:
test_x, test_y = test.iloc[:,:-1], test.iloc[:,-1]
new_network.evaluate(test_x, test_y)

3/3 [==============================] - 0s 614us/step - loss: 0.1760 - accuracy: 0.9600


[0.17600026726722717, 0.9599999785423279]

### Demo ends here
If you are reading it from `https://anonymous.4open.science/` there is a chance that the demo is duplicated.
Just ignore the following text.